# WIP Notebook

# Imports

## Libraries

In [1]:
%load_ext autoreload
%autoreload 2

In [79]:
# Standard Library Imports
import datetime
import glob
import io
import re
import warnings
import zipfile

# Third-Party Imports
import branca.colormap as cm
import folium
import geopandas as gpd
import holidays
import joblib
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
import pyod
from pyod.models.xgbod import IForest
import requests
import seaborn as sns
import shapely
from shapely import wkb
from shapely.geometry import Point

from scipy.stats import kurtosis, mode, pearsonr, skew, t
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from skrub import TableVectorizer

# Importing custom libraries
from wip_lib import date_encoder

## Data

### Provided data

In [113]:
# Import provided data
train = pd.read_parquet("/Users/pierrehaas/bike_counters/data/train.parquet")
test = pd.read_parquet("/Users/pierrehaas/bike_counters/data/final_test.parquet")

### External data

#### Weather data

In [112]:
# Import additionally sourced data

# https://meteo.data.gouv.fr/datasets/donnees-climatologiques-de-base-horaires/
weather = pd.read_csv(
    "/Users/pierrehaas/bike_counters/external_data/weather/H_75_previous-2020-2022.csv.gz",
    parse_dates=["AAAAMMJJHH"],
    date_format="%Y%m%d%H",
    compression="gzip",
    sep=";",
).rename(columns={"AAAAMMJJHH": "date"})

#### Public transport data

In [85]:
# https://data.iledefrance-mobilites.fr/explore/dataset/histo-validations-reseau-ferre/information/
# URLs of the zip files
urls = [
    "https://data.iledefrance-mobilites.fr/explore/dataset/histo-validations-reseau-ferre/files/e6bcf4c994951fc086e31db6819a3448/download/",
    "https://data.iledefrance-mobilites.fr/explore/dataset/histo-validations-reseau-ferre/files/e35b9ec0a183a8f2c7a8537dd43b124c/download/",
]

# Initialize an empty list to store DataFrames
dfs = []

# File matching pattern
pattern = r"data-rf-202\d/202\d_S\d+_NB_FER\.txt"

# Process each ZIP file
for url in urls:
    response = requests.get(url)
    if response.status_code == 200:
        with zipfile.ZipFile(io.BytesIO(response.content)) as z:
            # Get a list of all files in the archive and filter matching files
            matching_files = [f for f in z.namelist() if re.match(pattern, f)]

            # Read and concatenate the matching files
            for file in matching_files:
                with z.open(file) as f:
                    # Assuming the files are tab-separated and have a "JOUR" column
                    df = pd.read_csv(f, sep="\t", parse_dates=["JOUR"], dayfirst=True)
                    dfs.append(df)

# Combine all DataFrames
underground_transport = pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()


# # https://data.iledefrance-mobilites.fr/explore/dataset/histo-validations-reseau-surface/information/
# # URLs of the zip files
urls = [
    "https://data.iledefrance-mobilites.fr/explore/dataset/histo-validations-reseau-surface/files/41adcbd4216382c232ced4ccbf60187e/download/",
    "https://data.iledefrance-mobilites.fr/explore/dataset/histo-validations-reseau-surface/files/68cac32e8717f476905a60006a4dca26/download/",
]

# Initialize an empty list to store DataFrames
dfs = []

# File matching pattern
pattern = r"data-rs-202\d/202\d_T\d+_NB_SURFACE\.txt"

# Process each ZIP file
for url in urls:
    response = requests.get(url)
    if response.status_code == 200:
        with zipfile.ZipFile(io.BytesIO(response.content)) as z:
            # Get a list of all files in the archive and filter matching files
            matching_files = [f for f in z.namelist() if re.match(pattern, f)]

            # Read and concatenate the matching files
            for file in matching_files:
                with z.open(file) as f:
                    # Assuming the files are tab-separated and have a "JOUR" column
                    df = pd.read_csv(
                        f,
                        sep="\t",
                        parse_dates=["JOUR"],
                        dayfirst=True,
                        encoding="latin1",
                    )
                    dfs.append(df)

# Combine all DataFrames
overground_transport = pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()

#### Car traffic data

In [6]:
# URLs of the zip files
urls = [
    "https://parisdata.opendatasoft.com/api/datasets/1.0/comptages-routiers-permanents-historique/attachments/opendata_txt_2020_zip/",
    # "https://parisdata.opendatasoft.com/api/datasets/1.0/comptages-routiers-permanents-historique/attachments/opendata_txt_2021_zip/", # This file's compression format is broken, thus I provide a download link below
    "https://www.dropbox.com/scl/fi/sfqzlzpyxcf4yied3yucc/comptage-routier-2021.zip?rlkey=6k6hr3kywl8tvm4ax1qv2nv88&st=ktehiium&dl=1",
]

# Initialize an empty list to store DataFrames
dfs = []

# Process each ZIP file
for i, url in enumerate(urls):
    response = requests.get(url)
    if response.status_code == 200:
        with zipfile.ZipFile(io.BytesIO(response.content)) as z:
            # Process every file in the archive
            for file in z.namelist():
                # Skip directories and __MACOSX files
                if file.endswith("/") or "__MACOSX" in file:
                    continue
                # For the second URL, ensure files are within "comptage-routier-2021" directory
                if i == 1 and not file.startswith("comptage-routier-2021/"):
                    continue
                with z.open(file) as f:
                    # Assuming the files are semicolon-separated and have a "t_1h" column
                    df = pd.read_csv(f, sep=";", parse_dates=["t_1h"])
                    dfs.append(df)

# Combine all DataFrames
cars_count = pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()

#### Construction sites

In [176]:
# https://opendata.paris.fr/explore/dataset/chantiers-a-paris-copie0/information/?disjunctive.cp_arrondissement&disjunctive.chantier_categorie&disjunctive.moa_principal&disjunctive.chantier_synthese&disjunctive.localisation_detail&disjunctive.localisation_stationnement
construction_sites_2020 = pd.read_parquet(
    "/Users/pierrehaas/bike_counters/external_data/construction_sites/chantiers-a-paris-2020.parquet"
)

# https://opendata.paris.fr/explore/embed/dataset/chantiers-a-paris-copie1/table/?disjunctive.cp_arrondissement&disjunctive.chantier_categorie&disjunctive.moa_principal&disjunctive.chantier_synthese&disjunctive.localisation_detail&disjunctive.localisation_stationnement
construction_sites_2021 = pd.read_parquet(
    "/Users/pierrehaas/bike_counters/external_data/construction_sites/chantiers-a-paris-2021.parquet"
)

# EDA

## Exploring train and test sets

### Basic exploration

In [ ]:
train.head()

In [ ]:
train.shape

In [ ]:
train.info()

In [ ]:
train.nunique()

In [ ]:
train["date"].min(), train["date"].max()

In [ ]:
test.head()

In [ ]:
test.shape

In [ ]:
test.info()

In [ ]:
test.nunique()

In [ ]:
test["date"].min(), test["date"].max()

Checking installation dates

In [ ]:
train["counter_installation_date"].unique()

### Exploring bike count through time and location

#### Mapping of bike count by counter location

In [ ]:
# Calculate the total bike count per location
location_counts = (
    train.groupby(["latitude", "longitude"])["bike_count"].sum().reset_index()
)

# Create a colormap
colormap = cm.linear.PuRd_09.scale(
    location_counts["bike_count"].min(), location_counts["bike_count"].max()
)

# Create the map
m = folium.Map(
    location=train[["latitude", "longitude"]].mean(axis=0).values, zoom_start=13
)

# Add markers to the map
for _, row in location_counts.iterrows():
    folium.CircleMarker(
        location=[row["latitude"], row["longitude"]],
        radius=7,
        color=colormap(row["bike_count"]),
        fill=True,
        fill_color="black",  # Fill center with black
        fill_opacity=1,  # Ensure the fill is opaque
        popup=f"Bike Count: {row['bike_count']}",
    ).add_to(m)

# Add the colormap to the map
colormap.add_to(m)

m

It seems that the further away from the city center, the less bike passage there is. Let's try to find a smart way of creating a new feature that represents this.

In [ ]:
(
    train.groupby("site_name", observed=True)["bike_count"]
    .sum()
    .sort_values(ascending=False)
).head()

The counter with most bikes registered is: Totem 73 boulevard de Sébastopol. Let's use the coordinates from this site to create a distance feature.

#### Creating new distance feature

In [144]:
lat_long_center = (
    train[["latitude", "longitude"]]
    - mode(
        train[train["site_name"] == "Totem 73 boulevard de Sébastopol"][
            ["latitude", "longitude"]
        ]
    )[0]
)

# We calculate the distance the previously identified site to all others
# We call the result the distance to the center since the station is close to the center of Paris
dist_center = np.linalg.norm(
    lat_long_center,
    axis=1,
)

In [ ]:
print(
    f"Correlation coefficient (Distance to station with most counts VS. Log bike count): {pearsonr(dist_center, train["log_bike_count"])[0]:.3f}"
)

In [ ]:
plt.scatter(dist_center, train["log_bike_count"])
plt.show()

In [147]:
train["distance_to_center"] = dist_center

We repeat the same process for the test set.

In [148]:
lat_long_center = (
    test[["latitude", "longitude"]]
    - mode(
        test[test["site_name"] == "Totem 73 boulevard de Sébastopol"][
            ["latitude", "longitude"]
        ]
    )[0]
)

test["distance_to_center"] = np.linalg.norm(
    lat_long_center,
    axis=1,
)

#### Creating clusters of sites

In [149]:
# Group by counter_name and calculate the mean log_bike_count
grouped_train = (
    train.groupby("counter_name", observed=True)["log_bike_count"].mean().reset_index()
)

# Reshape the data for clustering
X = grouped_train[["log_bike_count"]]

# Apply K-Means clustering
kmeans = KMeans(n_clusters=5, random_state=42)  # You can change the number of clusters
grouped_train["cluster"] = kmeans.fit_predict(X)

# Sort clusters by mean log_bike_count and reassign cluster labels
sorted_clusters = (
    grouped_train.groupby("cluster")["log_bike_count"].mean().sort_values().index
)
cluster_mapping = {
    old_label: new_label for new_label, old_label in enumerate(sorted_clusters)
}
grouped_train["cluster"] = grouped_train["cluster"].map(cluster_mapping)

# Merge the cluster labels back to the original DataFrame
train = train.merge(
    grouped_train[["counter_name", "cluster"]], on="counter_name", how="left"
)
test = test.merge(
    grouped_train[["counter_name", "cluster"]], on="counter_name", how="left"
)

#### Time series of bike count by location

In [ ]:
# Pivot the DataFrame so each counter name has its own column
pivot_df = (
    train.groupby([pd.Grouper(key="date", freq="W"), "site_name"], observed=False)[
        "bike_count"
    ]
    .sum()
    .unstack()
)

# Plot the data
pivot_df.plot(figsize=(12, 6))
plt.title("Bike Count by Date")
plt.xlabel("Date")
plt.ylabel("Bike Count")
plt.legend(title="Site Name", bbox_to_anchor=(1.05, 1), loc="upper left")
plt.tight_layout()
plt.show()

#### Distribution of bike count by location

In [ ]:
# Create the boxplot
plt.figure(figsize=(12, 6))
sns.boxplot(x="counter_name", y="log_bike_count", data=train)

# Add labels and title
plt.xlabel("Counter Name")
plt.ylabel("Log Bike Count")
plt.title("Boxplot of Log Bike Count by Counter Name")

# Hide the x-axis labels
plt.xticks([])

# Display the plot
plt.show()

#### Weekly bike count table by location

In [ ]:
train.groupby([pd.Grouper(key="date", freq="W"), "site_name"], observed=False)[
    "bike_count"
].sum().unstack()

It seems like some sites are not capturing any data at all at times (multiple weeks sometimes). We will need to investigate this further.

Let's take a specific site that did not capture bikes for multiple weeks.

In [ ]:
# Set the style
sns.set(style="whitegrid")

# Filter the data
filtered_data = train[
    (train["site_name"] == "20 Avenue de Clichy") & (train["bike_count"] == 0)
]["date"].dt.hour

# Create the histogram
plt.figure(figsize=(10, 6))
sns.histplot(filtered_data, bins=24, kde=True, color="purple")

# Add titles and labels
plt.title(
    "Distribution of Hours with Zero Bike Count at 20 Avenue de Clichy", fontsize=16
)
plt.xlabel("Hour of the Day", fontsize=14)
plt.ylabel("Frequency", fontsize=14)

# Customize the ticks
plt.xticks(range(0, 24), fontsize=12)
plt.yticks(fontsize=12)

# Add gridlines
plt.grid(True, linestyle="--", alpha=0.7)

# Show the plot
plt.show()

We should try to find information about sites under repair (or else) to improve our model and make sure it predicts 0 bike_count when the site cannot capture data.

Let's check if there are changes to some variables (e.g., change in counter id) when the bike count is 0 for a prolonged period.

In [ ]:
train[
    (train["date"] >= "2021-05-09")
    & (train["date"] <= "2021-07-25")
    & (train["site_name"] == "20 Avenue de Clichy")
].groupby("counter_name", observed=True).nunique()

There does not seem to be a pattern of change in any variable when the bike count is 0 for a prolonged period.

### Encoding dates for further analyses

In [155]:
train_encoded, test_encoded = date_encoder(train), date_encoder(test)

In [ ]:
train_encoded.head()

In [ ]:
test_encoded.head()

### Analyzing impact of date features on bike count

In [ ]:
fig, axes = plt.subplots(3, 2, figsize=(15, 15))

# Plot for 'is_bank_holiday'
sns.boxplot(
    ax=axes[0, 0], x=train_encoded["is_bank_holiday"], y=train_encoded["log_bike_count"]
)
plt.setp(axes[0, 0].artists, alpha=0.5)
sns.violinplot(
    ax=axes[0, 0], x=train_encoded["is_bank_holiday"], y=train_encoded["log_bike_count"]
)
plt.setp(axes[0, 0].collections, alpha=0.5)
axes[0, 0].set_title("is_bank_holiday")
axes[0, 0].tick_params(axis="x", rotation=45)

# Plot for 'is_school_holiday'
sns.boxplot(
    ax=axes[0, 1],
    x=train_encoded["is_school_holiday"],
    y=train_encoded["log_bike_count"],
)
plt.setp(axes[0, 1].artists, alpha=0.5)
sns.violinplot(
    ax=axes[0, 1],
    x=train_encoded["is_school_holiday"],
    y=train_encoded["log_bike_count"],
)
plt.setp(axes[0, 1].collections, alpha=0.5)
axes[0, 1].set_title("is_school_holiday")
axes[0, 1].tick_params(axis="x", rotation=45)

# Plot for 'is_weekend'
sns.boxplot(
    ax=axes[1, 0], x=train_encoded["is_weekend"], y=train_encoded["log_bike_count"]
)
plt.setp(axes[1, 0].artists, alpha=0.5)
sns.violinplot(
    ax=axes[1, 0], x=train_encoded["is_weekend"], y=train_encoded["log_bike_count"]
)
plt.setp(axes[1, 0].collections, alpha=0.5)
axes[1, 0].set_title("is_weekend")
axes[1, 0].tick_params(axis="x", rotation=45)

# Plot for 'weekday'
sns.boxplot(
    ax=axes[1, 1], x=train_encoded["weekday"], y=train_encoded["log_bike_count"]
)
plt.setp(axes[1, 1].artists, alpha=0.5)
sns.violinplot(
    ax=axes[1, 1], x=train_encoded["weekday"], y=train_encoded["log_bike_count"]
)
plt.setp(axes[1, 1].collections, alpha=0.5)
axes[1, 1].set_title("weekday")
axes[1, 1].tick_params(axis="x", rotation=45)

# Plot for 'month'
sns.boxplot(ax=axes[2, 0], x=train_encoded["month"], y=train_encoded["log_bike_count"])
plt.setp(axes[2, 0].artists, alpha=0.5)
sns.violinplot(
    ax=axes[2, 0], x=train_encoded["month"], y=train_encoded["log_bike_count"]
)
plt.setp(axes[2, 0].collections, alpha=0.5)
axes[2, 0].set_title("month")
axes[2, 0].tick_params(axis="x", rotation=45)

# Plot for 'quarter'
sns.boxplot(
    ax=axes[2, 1], x=train_encoded["quarter"], y=train_encoded["log_bike_count"]
)
plt.setp(axes[2, 1].artists, alpha=0.5)
sns.violinplot(
    ax=axes[2, 1], x=train_encoded["quarter"], y=train_encoded["log_bike_count"]
)
plt.setp(axes[2, 1].collections, alpha=0.5)
axes[2, 1].set_title("quarter")

plt.tight_layout()
plt.show()

In [ ]:
# Create the plot
plot = (
    train_encoded.groupby(["hour", "weekday"])["log_bike_count"]
    .mean()
    .unstack()
    .plot(
        figsize=(12, 8),  # Increase the figure size
        colormap="viridis",  # Use a colormap
        title="Average Log Bike Count by Hour and Weekday",  # Add a title
    )
)

# Add labels
plot.set_xlabel("Hour")
plot.set_ylabel("Average Log Bike Count")
plot.set_title("Average Log Bike Count by Hour and Weekday")

# Add grid
plt.grid(True)

# Show the plot
plt.show()

In [ ]:
# Create the plot
plot = (
    train_encoded.groupby(["hour_scaled", "weekday"])["log_bike_count"]
    .mean()
    .unstack()
    .plot(
        figsize=(12, 8),  # Increase the figure size
        colormap="viridis",  # Use a colormap
        title="Average Log Bike Count by Hour and Weekday",  # Add a title
    )
)

# Add labels
plot.set_xlabel("Hour")
plot.set_ylabel("Average Log Bike Count")
plot.set_title("Average Log Bike Count by Hour and Weekday")

# Add grid
plt.grid(True)

# Show the plot
plt.show()

## Weather data

### Basic exploration

In [ ]:
weather.head()

In [ ]:
weather.shape

In [ ]:
weather.info()

In [ ]:
weather["date"].min(), weather["date"].max()

In [ ]:
weather[["NUM_POSTE", "NOM_USUEL", "LAT", "LON", "ALTI"]].nunique()

In [ ]:
weather.groupby("NUM_POSTE")["NOM_USUEL"].first()

Check the number of weather stations in the dataset.

In [ ]:
plt.figure(figsize=(8, 4))
plt.plot(weather.groupby("date")["NOM_USUEL"].count(), linestyle="-")
plt.title("Number of data collection stations per Date")
plt.xlabel("Date")
plt.ylabel("Count of NOM_USUEL")
plt.xticks(rotation=45)
plt.grid(False)
plt.show()

Now, let's have a look at the location of these weather stations.

In [ ]:
# Calculate the total count per location
location_counts = (
    weather.groupby(["LAT", "LON", "NUM_POSTE"]).size().reset_index(name="count")
)

# Create the map
m = folium.Map(location=weather[["LAT", "LON"]].mean(axis=0).values, zoom_start=13)

# Add markers to the map
for _, row in location_counts.iterrows():
    folium.CircleMarker(
        location=[row["LAT"], row["LON"]],
        radius=5,
        color="black",
        fill=True,
        fill_color="black",
        fill_opacity=1,
        popup=f"NUM_POSTE: {row['NUM_POSTE']}<br>Count: {row['count']}",
    ).add_to(m)

m

In our view, there are two potential approaches to merging the weather data with the train and test sets:

* Match the weather data to the train and test sets based on the closest weather station to the coordinates.
* Match the weather data to the train and test sets using the average weather information from all collection stations in Paris.

Let's check for missing values in the weather data. We shall drop the "NUM_POSTE", "NOM_USUEL", "LAT", "LON", "ALTI" columns since they are categorical (or just a marker of the specific data collection station).

In [ ]:
weather.drop(
    columns=["NUM_POSTE", "NOM_USUEL", "LAT", "LON", "ALTI", "date"]
).isna().sum()

There seem to be fields with no values at all. We shall drop these fields and continue a more detailed on the remaining fields.

In [ ]:
weather.drop(columns=["NUM_POSTE", "NOM_USUEL", "LAT", "LON", "ALTI", "date"]).dropna(
    axis=1, how="all"
).isna().sum()

We just went from 198 to 91 columns.

Since there are a lot of missing values in the weather data, we will start by check if there are collection stations that have more missing values than others.

In [ ]:
missing_values_per_num_poste = (
    weather.drop(columns=["NOM_USUEL", "LAT", "LON", "ALTI", "date"])
    .dropna(axis=1, how="all")
    .groupby("NUM_POSTE")
    .apply(lambda x: x.isna().sum(), include_groups=False)
)
missing_values_per_num_poste.sum(axis=1)

Maybe we should use the 75114001 station only, or just average out the values from all stations.

 Now, let's compute the average measure of each weather feature for each date in Paris as a whole (average out the measure over all stations at a given point in time).

In [ ]:
(
    weather.drop(columns=["NUM_POSTE", "NOM_USUEL", "LAT", "LON", "ALTI"])
    .dropna(axis=1, how="all")
    .groupby("date")
    .mean()
    .isna()
    .sum()
    .sort_values()
)

"QDXI3S" seems to have many missing values. We shall exclude it from the analysis. Now, let's check the remaining fields for missing values.

In [ ]:
(
    weather.drop(columns=["NUM_POSTE", "NOM_USUEL", "LAT", "LON", "ALTI", "QDXI3S"])
    .groupby("date")
    .mean()
    .dropna(axis=1, how="all")
    .isna()
    .sum()
    .sort_values()
    != 0
).sum()

There are 12 fields with missing values. We shall fill these missing values with linear interpolation since there are very few missing values. This is for sake of simplicity.

In [174]:
weather_reduced = (
    weather.drop(columns=["NUM_POSTE", "NOM_USUEL", "LAT", "LON", "QDXI3S"])
    .groupby("date")
    .mean()
    .dropna(axis=1, how="all")
    .interpolate(method="linear")
)

### Augmenting weather data with feature engineering

In [ ]:
weather_reduced.head()

In [176]:
weather_reduced["is_rain"] = (weather_reduced["RR1"] > 0).astype(int)

weather_reduced["q_rain_lag_1h"] = weather_reduced["RR1"].shift(1)
weather_reduced["t_rain_lag_1h"] = weather_reduced["DRR1"].shift(1)
weather_reduced["is_rain_lag_1h"] = weather_reduced["is_rain"].shift(1)

weather_reduced["q_rain_next_1h"] = weather_reduced["RR1"].shift(-1)
weather_reduced["t_rain_next_1h"] = weather_reduced["DRR1"].shift(-1)
weather_reduced["is_rain_next_1h"] = weather_reduced["is_rain"].shift(-1)

weather_reduced["temp_lag_1h"] = weather_reduced["T"].shift(1)
weather_reduced["temp_next_1h"] = weather_reduced["T"].shift(-1)

weather_reduced["max_temp"] = weather_reduced.groupby(weather_reduced.index.date)[
    "T"
].transform("max")
weather_reduced["will_rain"] = (
    weather_reduced.groupby(weather_reduced.index.date)["RR1"]
    .transform(lambda x: (x > 0).any())
    .astype(int)
)

### Exploring weather relationship with bike count

In [177]:
weather_eda = (
    train_encoded[["date", "counter_id", "log_bike_count"]]
    .merge(weather_reduced, left_on="date", right_on="date", how="left")
    .set_index(["date", "counter_id"])
)

#### Correlation heatmap, uncovering linear relationships

In [178]:
# Calculate the correlation matrix
correlation_matrix = weather_eda.corr()

In [ ]:
# Set up the matplotlib figure
plt.figure(figsize=(14, 14))

# Draw the heatmap with the correlation matrix
sns.heatmap(
    correlation_matrix,
    annot=False,
    cmap="coolwarm",
)

# Set the title
plt.title("Correlation Matrix")

# Display the plot
plt.show()

In [ ]:
threshold = 0.1

high_corr_features = correlation_matrix[
    np.abs(correlation_matrix["log_bike_count"]) > threshold
].index.values.tolist()

# Set up the matplotlib figure
plt.figure(figsize=(10, 8))

# Draw the heatmap with the correlation matrix
sns.heatmap(
    correlation_matrix[np.abs(correlation_matrix["log_bike_count"]) > threshold][
        high_corr_features
    ],
    annot=False,
    cmap="coolwarm",
)

# Set the title
plt.title("Correlation Matrix")

# Display the plot
plt.show()

#### Exploring relationships of selected features with log_bike_count

In [181]:
weather_features = [
    "RR1",
    "DRR1",
    "T",
    "TNSOL",
    "TCHAUSSEE",
    "U",
    "GLO",
    "is_rain",
    "q_rain_lag_1h",
    "t_rain_lag_1h",
    "is_rain_lag_1h",
    "q_rain_next_1h",
    "t_rain_next_1h",
    "is_rain_next_1h",
    "temp_lag_1h",
    "temp_next_1h",
    "max_temp",
    "will_rain",
]

In [182]:
weather_eda = (
    train_encoded[["log_bike_count", "date", "hour", "weekday", "counter_id"]]
    .merge(
        weather_reduced[weather_features], left_on="date", right_on="date", how="left"
    )
    .set_index(["date", "counter_id"])
)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

# Plot for 'is_rain'
sns.boxplot(ax=ax, x=weather_eda["is_rain"], y=weather_eda["log_bike_count"])
plt.setp(ax.artists, alpha=0.5)
sns.violinplot(ax=ax, x=weather_eda["is_rain"], y=weather_eda["log_bike_count"])
plt.setp(ax.collections, alpha=0.5)
ax.set_title("is_rain")
ax.tick_params(axis="x", rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Create subplots
fig, axes = plt.subplots(2, 4, figsize=(20, 12), sharey=True)

# Flatten the axes array for easy iteration
axes = axes.flatten()

# List of weekdays
weekdays = [
    "Monday",
    "Tuesday",
    "Wednesday",
    "Thursday",
    "Friday",
    "Saturday",
    "Sunday",
]

# Plot each weekday
for i, weekday in enumerate(range(1, 8)):
    ax = axes[i]
    sns.boxplot(
        ax=ax,
        x=weather_eda["is_rain"],
        y=weather_eda["log_bike_count"],
        data=weather_eda[weather_eda["weekday"] == weekday],
    )
    plt.setp(ax.artists, alpha=0.5)
    sns.violinplot(
        ax=ax,
        x=weather_eda["is_rain"],
        y=weather_eda["log_bike_count"],
        data=weather_eda[weather_eda["weekday"] == weekday],
    )
    plt.setp(ax.collections, alpha=0.5)
    ax.set_title(weekdays[i])
    ax.tick_params(axis="x", rotation=45)

# Remove any unused subplots
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

In [ ]:
# Create the plot
plt.figure(figsize=(14, 8))

# Plot for is_rain = 0
sns.lineplot(
    data=weather_eda[weather_eda["is_rain"] == 0],
    x="weekday",
    y="log_bike_count",
    label="No Rain",
)

# Plot for is_rain = 1
sns.lineplot(
    data=weather_eda[weather_eda["is_rain"] == 1],
    x="weekday",
    y="log_bike_count",
    label="Rain",
)

# Add labels and title
plt.xlabel("Weekday")
plt.ylabel("Log Bike Count")
plt.title("Log Bike Count by Day and Rain Condition")
plt.legend(title="Rain Condition")

# Show the plot
plt.show()

In [ ]:
# Number of features
num_features = len(weather_features)

# Calculate the number of rows and columns for subplots
nrows = (num_features + 1) // 2
ncols = 2

# Create subplots
fig, axes = plt.subplots(nrows, ncols, figsize=(15, 20))

# Flatten the axes array for easy iteration
axes = axes.flatten()

# Plot each feature
for i, feature in enumerate(weather_features):
    axes[i].scatter(weather_eda[feature], weather_eda["log_bike_count"])
    axes[i].set_xlabel(feature)
    axes[i].set_ylabel("Log Bike Count")
    axes[i].set_title(f"{feature} vs Log Bike Count")

# Remove any unused subplots
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

# Adjust layout
plt.tight_layout()
plt.show()

In [ ]:
# Calculate the correlation matrix
correlation_matrix = weather_eda.corr()

# Set up the matplotlib figure
plt.figure(figsize=(10, 8))

# Draw the heatmap with the correlation matrix
sns.heatmap(
    correlation_matrix,
    annot=False,
    cmap="coolwarm",
)

# Set the title
plt.title("Correlation Matrix")

# Display the plot
plt.show()

### Reducing dimensionality of weather data

In [189]:
weather_features = [
    "RR1",
    "DRR1",
    "T",
    "TNSOL",
    "TCHAUSSEE",
    "U",
    "GLO",
    # "is_rain",
    "q_rain_lag_1h",
    "t_rain_lag_1h",
    # "is_rain_lag_1h",
    "q_rain_next_1h",
    "t_rain_next_1h",
    # "is_rain_next_1h",
    "temp_lag_1h",
    "temp_next_1h",
    "max_temp",
    # "will_rain",
]

In [191]:
mask = (weather_reduced.index >= train["date"].min()) & (
    weather_reduced.index <= test["date"].max()
)

In [ ]:
# Perform PCA with the maximum number of components
pca = PCA()
pca.fit(weather_reduced[mask][weather_features])

# Calculate the cumulative explained variance ratio
cumulative_explained_variance = pca.explained_variance_ratio_.cumsum()

# Plot the cumulative explained variance ratio
plt.figure(figsize=(10, 6))
plt.plot(
    range(1, len(cumulative_explained_variance) + 1),
    cumulative_explained_variance,
    marker="o",
    linestyle="--",
    color="b",
)
plt.title("Explained Variance by Number of PCA Components")
plt.xlabel("Number of Components")
plt.ylabel("Cumulative Explained Variance")
plt.grid(True)
plt.show()

In [ ]:
n = 5

pca = PCA(n_components=n)

X_pca1 = pca.fit_transform(weather_reduced[mask][weather_features])

weather_pca = pd.DataFrame(
    X_pca1,
    index=weather_reduced[mask][weather_features].index,
    columns=["weather_" + str(i) for i in range(1, n + 1)],
).reset_index()

print(
    f"% of data variance explained by reduced PCA with {n} components: {pca.explained_variance_ratio_.sum():.3f}"
)

In [ ]:
weather_pca.head()

### Merging weather data with train and test sets

We propose to ways of merging the weather data with the train and test sets:

* Merge handpicked weather features with the train and test sets.

In [70]:
# train_encoded = train_encoded.merge(
#     weather_reduced[weather_features],
#     left_on="date",
#     right_index=True,
#     how="left",
# )

# test_encoded = test_encoded.merge(
#     weather_reduced[weather_features],
#     left_on="date",
#     right_index=True,
#     how="left",
# )

* Merge the PCA-transformed weather data with the train and test sets.

In [71]:
train_encoded = train_encoded.merge(
    weather_pca,
    on="date",
    how="left",
)

test_encoded = test_encoded.merge(
    weather_pca,
    on="date",
    how="left",
)

## Public transport data

### Basic exploration

Let's compute the total number of commuters per day in Ile-de-France.

In [ ]:
daily_underground_transport = underground_transport.groupby("JOUR")["NB_VALD"].sum()
daily_underground_transport

In [ ]:
daily_overground_transport = overground_transport.groupby("JOUR")["NB_VALD"].sum()
daily_overground_transport

In [ ]:
# Calculate daily totals
daily_underground_transport = underground_transport.groupby("JOUR")["NB_VALD"].sum()
daily_overground_transport = overground_transport.groupby("JOUR")["NB_VALD"].sum()

# Resample to weekly totals
weekly_underground_transport = daily_underground_transport.resample("W").sum()
weekly_overground_transport = daily_overground_transport.resample("W").sum()

# Plot daily totals
plt.figure(figsize=(12, 6))
plt.subplot(2, 1, 1)
plt.plot(daily_underground_transport, label="Underground")
plt.plot(daily_overground_transport, label="Overground")
plt.plot(daily_underground_transport + daily_overground_transport, label="Total")
plt.legend()
plt.title("Daily Transport Totals")

# Plot weekly totals
plt.subplot(2, 1, 2)
plt.plot(weekly_underground_transport, label="Underground")
plt.plot(weekly_overground_transport, label="Overground")
plt.plot(weekly_underground_transport + weekly_overground_transport, label="Total")
plt.legend()
plt.title("Weekly Transport Totals")

plt.tight_layout()
plt.show()

Let's combine the daily number of commuters of both overground and underground transport and encode the dates to push further analysis.

In [75]:
daily_transport = (
    daily_underground_transport + daily_overground_transport
).reset_index()

daily_transport = date_encoder(daily_transport, col="JOUR")[
    (daily_transport["JOUR"] >= "2020-09-01")
    & (daily_transport["JOUR"] <= "2021-10-18")
]

In [ ]:
daily_transport.head()

Using this new data, we will try to detect days of strikes in the public transport system.

### Outlier detection

In [ ]:
plt.hist(daily_transport["NB_VALD"])
plt.show()

In [ ]:
# Create a fancy boxplot
fig, ax = plt.subplots(figsize=(6, 6))

# Plot boxplot for 'NB_VALD'
sns.boxplot(ax=ax, y=daily_transport["NB_VALD"])
plt.setp(ax.artists, alpha=0.5)

# Optionally, add a violin plot for additional visualization
sns.violinplot(ax=ax, y=daily_transport["NB_VALD"])
plt.setp(ax.collections, alpha=0.5)

ax.set_title("Boxplot of NB_VALD", fontsize=16)
ax.set_xlabel("NB_VALD", fontsize=14)
ax.tick_params(axis="x", rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Create a fancy boxplot
fig, ax = plt.subplots(figsize=(6, 6))

# Plot boxplot for 'NB_VALD'
sns.boxplot(ax=ax, y=daily_transport["NB_VALD"])
plt.setp(ax.artists, alpha=0.5)

# Optionally, add a violin plot for additional visualization
sns.violinplot(ax=ax, y=daily_transport["NB_VALD"])
plt.setp(ax.collections, alpha=0.5)

ax.set_title("Boxplot of NB_VALD", fontsize=16)
ax.set_xlabel("NB_VALD", fontsize=14)
ax.tick_params(axis="x", rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
avg_daily_transport = daily_transport["NB_VALD"].mean()
std_daily_transport = daily_transport["NB_VALD"].std()
mask = (daily_transport["NB_VALD"] < avg_daily_transport - 2 * std_daily_transport) | (
    daily_transport["NB_VALD"] > avg_daily_transport + 2 * std_daily_transport
)

daily_transport[mask]

### Merging public transport data with train and test sets

In [81]:
train_encoded = train_encoded.merge(
    daily_transport[["year", "month", "day", "NB_VALD"]],
    left_on=["year", "month", "day"],
    right_on=["year", "month", "day"],
    how="left",
)

test_encoded = test_encoded.merge(
    daily_transport[["year", "month", "day", "NB_VALD"]],
    left_on=["year", "month", "day"],
    right_on=["year", "month", "day"],
    how="left",
)

## Car traffic data

### Basic exploration

In [ ]:
cars_count.head()

In [ ]:
hourly_cars_count = (
    cars_count[
        (cars_count["t_1h"] >= train["date"].min() - datetime.timedelta(hours=1))
        & (cars_count["t_1h"] <= test["date"].max() + datetime.timedelta(hours=1))
    ]
    .groupby("t_1h")["q"]
    .sum()
    .reset_index()
)

hourly_cars_count

In [ ]:
# Group by day and calculate the cumulative sum of 'q' for each day
hourly_cars_count["daily_cumsum"] = hourly_cars_count.groupby(
    hourly_cars_count["t_1h"].dt.to_period("d")
)["q"].cumsum()
hourly_cars_count.head()

In [ ]:
# Group by week and sum the car counts for each week
weekly_sum = (
    cars_count.groupby(cars_count["t_1h"].dt.to_period("W"))["q"].sum().reset_index()
)

# Convert the period to datetime for plotting
weekly_sum["t_1h"] = weekly_sum["t_1h"].dt.start_time

# Plot the weekly average number of cars
fig, ax = plt.subplots(figsize=(10, 6))
sns.lineplot(ax=ax, x=weekly_sum["t_1h"], y=weekly_sum["q"], marker="o")
ax.set_title("Weekly Average Number of Cars", fontsize=16)
ax.set_xlabel("Week", fontsize=14)
ax.set_ylabel("Average Number of Cars", fontsize=14)
ax.tick_params(axis="x", rotation=45)

plt.tight_layout()
plt.show()

### Merging car traffic data with train and test sets

In [86]:
train_encoded = train_encoded.merge(
    hourly_cars_count, left_on="date", right_on="t_1h", how="left"
)
test_encoded = test_encoded.merge(
    hourly_cars_count, left_on="date", right_on="t_1h", how="left"
)

## Construction sites

### Basic exploration

In [ ]:
construction_sites_2020.head()

In [ ]:
construction_sites_2020.shape

In [ ]:
construction_sites_2021.head()

In [ ]:
construction_sites_2021.shape

In [355]:
construction_sites = pd.concat(
    [construction_sites_2020, construction_sites_2021], axis=0
)

In [ ]:
construction_sites.head()

In [ ]:
construction_sites.shape

In [ ]:
construction_sites.info()

In [ ]:
construction_sites[
    [
        "num_emprise",
        "moa_principal",
        "demande_cite_id",
        "chantier_cite_id",
        "geo_shape",
        "geo_point_2d",
    ]
].nunique()

### Encoding dates to datetime

In [360]:
# construction_sites["date_debut"] = pd.to_datetime(
#     construction_sites["date_debut"], format="%Y-%m-%d"
# )

# construction_sites["date_fin"] = pd.to_datetime(
#     construction_sites["date_fin"], format="%Y-%m-%d"
# )

In [361]:
# construction_sites.info()

### Deleting duplicate construction sites and sites with no location information

Let's try to understand the data better by checking observations with duplicated chantier_cite_id.

In [ ]:
construction_sites[
    construction_sites["chantier_cite_id"].duplicated(keep=False)
].sort_values("chantier_cite_id").head()

It seems that the duplicated observations are not exactly the same. Looking at the first two observations, we can see that, although the chantier_cite_id are the same for the two, the observtions have different areas. We shall keep the two observations, and for that matter any other duplicated observation under the same scenario, and only drop duplicated observations that have the exact same information (it implies that the construction started in 2020 or before and ended in 2021 or after | observations were both in the 2020 and 2021 datasets).

In [363]:
construction_sites = construction_sites.drop_duplicates().dropna(
    subset=["geo_shape", "geo_point_2d"]
)

In [ ]:
construction_sites.shape

In [ ]:
has_duplicates = construction_sites.duplicated(keep=False).any()
print(f"Are there duplicates? {has_duplicates}")

In [ ]:
num_duplicates = construction_sites.duplicated(keep=False).sum()
print(f"Number of duplicated rows: {num_duplicates}")

### Selecting relevant construction sites

Let's take a look at the types of construction sites in the dataset.

In [ ]:
construction_sites["localisation_detail"].unique()

It seems like there is a special encoding for construction sites that are located ob bike lanes (PISTE_CYCLABLE). We shall take a look at these observations specifically.

In [368]:
# construction_sites_simplified = construction_sites[
#     ~construction_sites["localisation_detail"].isin(
#         [
#             "EMPRISE_CHAUSSEE",
#             "EMPRISE_TROTTOIR",
#             "EMPRISE_CHAUSSEE,EMPRISE_TROTTOIR",
#             None,
#         ]
#     )
# ]


# construction_sites_simplified["localisation_detail"].unique()

We drop irrelevant columns.

In [ ]:
cs_reduced = construction_sites.drop(
    columns=[
        "moa_principal",
        "demande_cite_id",
        "chantier_synthese",
        "localisation_stationnement",
        "chantier_cite_id",
        "chantier_categorie",
        "cp_arrondissement",
        "surface",
        "localisation_detail",
    ]
)

cs_reduced.head()

### Visualizing construction sites shape

In [ ]:
geo_point_wkb = cs_reduced["geo_shape"].iloc[0]
geometry = wkb.loads(geo_point_wkb)
print(geometry)

In [ ]:
# Extract x and y coordinates for plotting
x, y = geometry.exterior.xy

# Plot the polygon
plt.plot(x, y)
plt.title("Polygon Visualization")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()

### Transforming geoshapes to coordinates

In [372]:
cs_reduced[["geo_shape", "geo_point_2d"]] = cs_reduced[
    ["geo_shape", "geo_point_2d"]
].apply(lambda x: shapely.wkb.loads(x))

In [ ]:
null_count = cs_reduced["geo_shape"].isnull().sum()
if null_count > 0:
    print(f"There are {null_count} null geometries.")
else:
    print("No null geometries found.")

In [ ]:
null_count = cs_reduced["geo_point_2d"].isnull().sum()
if null_count > 0:
    print(f"There are {null_count} null geometries.")
else:
    print("No null geometries found.")

In [375]:
gdf = gpd.GeoDataFrame(cs_reduced, geometry="geo_shape", crs="EPSG:4326")

In [376]:
# gdf.explore("area", legend=False)

### Finding if bike counters fall within construction sites

In [377]:
latitude_combined = pd.concat(
    [train_encoded["latitude"], test_encoded["latitude"]], ignore_index=True
)
longitude_combined = pd.concat(
    [train_encoded["longitude"], test_encoded["longitude"]], ignore_index=True
)
coords = list(
    zip(
        latitude_combined,
        longitude_combined,
    )
)

index1 = pd.concat([train_encoded["date"], test_encoded["date"]], ignore_index=True)
index2 = pd.concat(
    [train_encoded["counter_id"], test_encoded["counter_id"]], ignore_index=True
)

points_gdf = gpd.GeoDataFrame(
    coords,
    index=[index1, index2],
    columns=["longitude", "latitude"],
    geometry=[Point(xy) for xy in coords],
    crs=gdf.crs,  # Make sure this matches the gdf of construction sites
)

points_gdf.head()

In [380]:
# spatial join with predicate='within' to see which points lie inside a polygon
joined = gpd.sjoin(points_gdf, gdf, how="left", predicate="within")

joined.head()

In [ ]:
print(
    f"Number of bike counters that fall (at some point) in a construction site: {joined.shape[0] - joined["num_emprise"].isna().sum()}"
)

There are no bike counters that fall within construction sites. Thus, the construction site dataset will not be useful for our analysis.

# Detecting outliers and potential strikes in public transport data

## Underground transport

### Using z-score

#### At station level

In [ ]:
ut_grouped = (
    underground_transport.groupby(["JOUR", "CODE_STIF_ARRET"])["NB_VALD"]
    .sum()
    .reset_index()
)
ut_grouped = ut_grouped[
    (ut_grouped["JOUR"] >= "2020-09-01") & (ut_grouped["JOUR"] <= "2021-10-18")
]

# Ensure JOUR is a datetime type
ut_grouped["JOUR"] = pd.to_datetime(ut_grouped["JOUR"])

# Extract the day of week (0=Monday, 6=Sunday)
ut_grouped["day_of_week"] = ut_grouped["JOUR"].dt.dayofweek

# Compute baseline stats (mean and std) per day_of_week
baseline_stats = (
    ut_grouped.groupby(["CODE_STIF_ARRET", "day_of_week"])["NB_VALD"]
    .agg(["mean", "std"])
    .reset_index()
)
baseline_stats.rename(columns={"mean": "mean_val", "std": "std_val"}, inplace=True)

# Merge the baseline stats back to the original DataFrame
ut_grouped = ut_grouped.merge(
    baseline_stats, on=["CODE_STIF_ARRET", "day_of_week"], how="left"
)

# Compute z-score for each day’s NB_VALD
# Handle cases where std is zero by replacing with a small number
ut_grouped["std_val"] = ut_grouped["std_val"].replace(0, np.nan)
ut_grouped["z_score"] = (ut_grouped["NB_VALD"] - ut_grouped["mean_val"]) / ut_grouped[
    "std_val"
]

# Define an anomaly threshold
# For example, we consider a strike if it's 3 standard deviations below the mean
threshold = -3

# Flag potential strike days
ut_grouped["potential_strike"] = ut_grouped["z_score"] < threshold

# Optionally, select only those rows that are flagged
possible_strike_days = ut_grouped[ut_grouped["potential_strike"]]

print("Potential strike days:")
possible_strike_days[
    ["JOUR", "CODE_STIF_ARRET", "NB_VALD", "z_score", "potential_strike"]
]

In [ ]:
possible_strike_days.groupby("JOUR")["potential_strike"].sum()

#### At daily level

In [ ]:
ut_grouped = underground_transport.groupby(["JOUR"])["NB_VALD"].sum().reset_index()
ut_grouped = ut_grouped[
    (ut_grouped["JOUR"] >= "2020-09-01") & (ut_grouped["JOUR"] <= "2021-10-18")
]

# Ensure JOUR is a datetime type
ut_grouped["JOUR"] = pd.to_datetime(ut_grouped["JOUR"])

# Extract the day of week (0=Monday, 6=Sunday)
ut_grouped["day_of_week"] = ut_grouped["JOUR"].dt.dayofweek

# Compute baseline stats (mean and std) per day_of_week
baseline_stats = (
    ut_grouped.groupby(["day_of_week"])["NB_VALD"].agg(["mean", "std"]).reset_index()
)
baseline_stats.rename(columns={"mean": "mean_val", "std": "std_val"}, inplace=True)

# Merge the baseline stats back to the original DataFrame
ut_grouped = ut_grouped.merge(baseline_stats, on=["day_of_week"], how="left")

# Compute z-score for each day’s NB_VALD
# Handle cases where std is zero by replacing with a small number
ut_grouped["std_val"] = ut_grouped["std_val"].replace(0, np.nan)
ut_grouped["z_score"] = (ut_grouped["NB_VALD"] - ut_grouped["mean_val"]) / ut_grouped[
    "std_val"
]

# Define an anomaly threshold
# For example, we consider a strike if it's 3 standard deviations below the mean
threshold = -1.5

# Flag potential strike days
ut_grouped["potential_strike"] = ut_grouped["z_score"] < threshold

# Optionally, select only those rows that are flagged
possible_strike_days = ut_grouped[ut_grouped["potential_strike"]]

print("Potential strike days:")
possible_strike_days[["JOUR", "NB_VALD", "z_score", "potential_strike"]]

In [ ]:
possible_strike_days.groupby("JOUR")["potential_strike"].sum()

### Using t-score

#### At station level

In [ ]:
# Group by day and station, sum NB_VALD
ut_grouped = (
    underground_transport.groupby(["JOUR", "CODE_STIF_ARRET"])["NB_VALD"]
    .sum()
    .reset_index()
)
ut_grouped = ut_grouped[
    (ut_grouped["JOUR"] >= "2020-09-01") & (ut_grouped["JOUR"] <= "2021-10-18")
]

# Ensure JOUR is a datetime type
ut_grouped["JOUR"] = pd.to_datetime(ut_grouped["JOUR"])

# Extract the day of week (0=Monday, 6=Sunday)
ut_grouped["day_of_week"] = ut_grouped["JOUR"].dt.dayofweek

# Compute baseline stats (mean, std, and count) per day_of_week and station
baseline_stats = (
    ut_grouped.groupby(["CODE_STIF_ARRET", "day_of_week"])["NB_VALD"]
    .agg(["mean", "std", "count"])
    .reset_index()
)
baseline_stats.rename(
    columns={"mean": "mean_val", "std": "std_val", "count": "n"}, inplace=True
)

# Merge the baseline stats back to the original DataFrame
ut_grouped = ut_grouped.merge(
    baseline_stats, on=["CODE_STIF_ARRET", "day_of_week"], how="left"
)

# Compute studentized residuals for each day’s NB_VALD
ut_grouped["std_val"] = ut_grouped["std_val"].replace(0, np.nan)
ut_grouped["studentized_residual"] = (
    (ut_grouped["NB_VALD"] - ut_grouped["mean_val"]) / ut_grouped["std_val"]
) * np.sqrt((ut_grouped["n"] + 1) / ut_grouped["n"])

# Define an anomaly threshold using the t-distribution
threshold = t.ppf(
    0.025, df=ut_grouped["n"] - 1
)  # 2.5% significance level for two-tailed test

# Flag potential strike days
ut_grouped["potential_strike"] = ut_grouped["studentized_residual"] < threshold

# Optionally, select only those rows that are flagged
possible_strike_days = ut_grouped[ut_grouped["potential_strike"]]

print("Potential strike days:")
possible_strike_days[
    ["JOUR", "CODE_STIF_ARRET", "NB_VALD", "studentized_residual", "potential_strike"]
]

In [ ]:
possible_strike_days.groupby("JOUR")["potential_strike"].sum()

#### At daily level

In [ ]:
# Group by day and sum NB_VALD
ut_grouped = underground_transport.groupby(["JOUR"])["NB_VALD"].sum().reset_index()
ut_grouped = ut_grouped[
    (ut_grouped["JOUR"] >= "2020-09-01") & (ut_grouped["JOUR"] <= "2021-10-18")
]

# Ensure JOUR is a datetime type
ut_grouped["JOUR"] = pd.to_datetime(ut_grouped["JOUR"])

# Extract the day of week (0=Monday, 6=Sunday)
ut_grouped["day_of_week"] = ut_grouped["JOUR"].dt.dayofweek

# Compute baseline stats (mean and std) per day_of_week
baseline_stats = (
    ut_grouped.groupby(["day_of_week"])["NB_VALD"]
    .agg(["mean", "std", "count"])
    .reset_index()
)
baseline_stats.rename(
    columns={"mean": "mean_val", "std": "std_val", "count": "n"}, inplace=True
)

# Merge the baseline stats back to the original DataFrame
ut_grouped = ut_grouped.merge(baseline_stats, on=["day_of_week"], how="left")

# Compute studentized residuals for each day’s NB_VALD
ut_grouped["std_val"] = ut_grouped["std_val"].replace(0, np.nan)
ut_grouped["studentized_residual"] = (
    (ut_grouped["NB_VALD"] - ut_grouped["mean_val"]) / ut_grouped["std_val"]
) * np.sqrt((ut_grouped["n"] + 1) / ut_grouped["n"])

# Define an anomaly threshold using the t-distribution
threshold = t.ppf(
    0.025, df=ut_grouped["n"] - 1
)  # 2.5% significance level for two-tailed test

# Flag potential strike days
ut_grouped["potential_strike"] = ut_grouped["studentized_residual"] < threshold

# Optionally, select only those rows that are flagged
possible_strike_days = ut_grouped[ut_grouped["potential_strike"]]

print("Potential strike days:")
possible_strike_days[["JOUR", "NB_VALD", "studentized_residual", "potential_strike"]]

In [ ]:
possible_strike_days.groupby("JOUR")["potential_strike"].sum()

### Using pyod library

In [ ]:
underground_transport.head()

In [ ]:
underground_transport.nunique()

In [96]:
# ut_grouped = underground_transport.groupby(["JOUR"])["NB_VALD"].sum().reset_index()
# ut_grouped = date_encoder(ut_grouped, col="JOUR")
# ut_grouped = ut_grouped[
#     (ut_grouped["JOUR"] >= "2020-09-01")
#     & (ut_grouped["JOUR"] <= "2021-10-18")
# ]

In [ ]:
ut_grouped.head()

In [98]:
# encoder = TableVectorizer()
# X_encoded = encoder.fit_transform(ut_grouped.drop(columns=["JOUR"]))
# X_encoded.head()

In [99]:
# model = IForest()
# model.fit(X_encoded)
# y_pred = model.predict(X_encoded)

In [100]:
# ut_grouped["outliers"] = y_pred

In [101]:
# ut_grouped.head()

In [102]:
# # Assuming 'outliers' DataFrame has a 'group' column for grouping and 'NB_VALD' column for values
# plt.figure(figsize=(12, 8))
# sns.boxplot(data=ut_grouped, x='outliers', y='NB_VALD')
# plt.title("Boxplot of NB_VALD by Group", fontsize=16)
# plt.xlabel("Group", fontsize=14)
# plt.ylabel("NB_VALD", fontsize=14)
# plt.xticks(rotation=45)
# plt.grid(True, linestyle='--', alpha=0.7)
# plt.tight_layout()
# plt.show()

## Overground transport

### Using z-score

#### At station level

In [ ]:
ot_grouped = (
    overground_transport.groupby(["JOUR", "CODE_STIF_LIGNE"])["NB_VALD"]
    .sum()
    .reset_index()
)
ot_grouped = ot_grouped[
    (ot_grouped["JOUR"] >= "2020-09-01") & (ot_grouped["JOUR"] <= "2021-10-18")
]

# Ensure JOUR is a datetime type
ot_grouped["JOUR"] = pd.to_datetime(ot_grouped["JOUR"])

# Extract the day of week (0=Monday, 6=Sunday)
ot_grouped["day_of_week"] = ot_grouped["JOUR"].dt.dayofweek

# Compute baseline stats (mean and std) per day_of_week
baseline_stats = (
    ot_grouped.groupby(["CODE_STIF_LIGNE", "day_of_week"])["NB_VALD"]
    .agg(["mean", "std"])
    .reset_index()
)
baseline_stats.rename(columns={"mean": "mean_val", "std": "std_val"}, inplace=True)

# Merge the baseline stats back to the original DataFrame
ot_grouped = ot_grouped.merge(
    baseline_stats, on=["CODE_STIF_LIGNE", "day_of_week"], how="left"
)

# Compute z-score for each day’s NB_VALD
# Handle cases where std is zero by replacing with a small number
ot_grouped["std_val"] = ot_grouped["std_val"].replace(0, np.nan)
ot_grouped["z_score"] = (ot_grouped["NB_VALD"] - ot_grouped["mean_val"]) / ot_grouped[
    "std_val"
]

# Define an anomaly threshold
# For example, we consider a strike if it's 3 standard deviations below the mean
threshold = -3

# Flag potential strike days
ot_grouped["potential_strike"] = ot_grouped["z_score"] < threshold

# Optionally, select only those rows that are flagged
possible_strike_days = ot_grouped[ot_grouped["potential_strike"]]

print("Potential strike days:")
possible_strike_days[
    ["JOUR", "CODE_STIF_LIGNE", "NB_VALD", "z_score", "potential_strike"]
]

In [ ]:
possible_strike_days.groupby("JOUR")["potential_strike"].sum()

#### At daily level

In [ ]:
ot_grouped = overground_transport.groupby(["JOUR"])["NB_VALD"].sum().reset_index()
ot_grouped = ot_grouped[
    (ot_grouped["JOUR"] >= "2020-09-01") & (ot_grouped["JOUR"] <= "2021-10-18")
]

# Ensure JOUR is a datetime type
ot_grouped["JOUR"] = pd.to_datetime(ot_grouped["JOUR"])

# Extract the day of week (0=Monday, 6=Sunday)
ot_grouped["day_of_week"] = ot_grouped["JOUR"].dt.dayofweek

# Compute baseline stats (mean and std) per day_of_week
baseline_stats = (
    ot_grouped.groupby(["day_of_week"])["NB_VALD"].agg(["mean", "std"]).reset_index()
)
baseline_stats.rename(columns={"mean": "mean_val", "std": "std_val"}, inplace=True)

# Merge the baseline stats back to the original DataFrame
ot_grouped = ot_grouped.merge(baseline_stats, on=["day_of_week"], how="left")

# Compute z-score for each day’s NB_VALD
# Handle cases where std is zero by replacing with a small number
ot_grouped["std_val"] = ot_grouped["std_val"].replace(0, np.nan)
ot_grouped["z_score"] = (ot_grouped["NB_VALD"] - ot_grouped["mean_val"]) / ot_grouped[
    "std_val"
]

# Define an anomaly threshold
# For example, we consider a strike if it's 3 standard deviations below the mean
threshold = -3

# Flag potential strike days
ot_grouped["potential_strike"] = ot_grouped["z_score"] < threshold

# Optionally, select only those rows that are flagged
possible_strike_days = ot_grouped[ot_grouped["potential_strike"]]

print("Potential strike days:")
possible_strike_days[["JOUR", "NB_VALD", "z_score", "potential_strike"]]

In [ ]:
possible_strike_days.groupby("JOUR")["potential_strike"].sum()

### Using t-score

#### At station level

In [ ]:
# Group by day and station, sum NB_VALD
ot_grouped = (
    overground_transport.groupby(["JOUR", "CODE_STIF_LIGNE"])["NB_VALD"]
    .sum()
    .reset_index()
)
ot_grouped = ot_grouped[
    (ot_grouped["JOUR"] >= "2020-09-01") & (ot_grouped["JOUR"] <= "2021-10-18")
]

# Ensure JOUR is a datetime type
ot_grouped["JOUR"] = pd.to_datetime(ot_grouped["JOUR"])

# Extract the day of week (0=Monday, 6=Sunday)
ot_grouped["day_of_week"] = ot_grouped["JOUR"].dt.dayofweek

# Compute baseline stats (mean, std, and count) per day_of_week and station
baseline_stats = (
    ot_grouped.groupby(["CODE_STIF_LIGNE", "day_of_week"])["NB_VALD"]
    .agg(["mean", "std", "count"])
    .reset_index()
)
baseline_stats.rename(
    columns={"mean": "mean_val", "std": "std_val", "count": "n"}, inplace=True
)

# Merge the baseline stats back to the original DataFrame
ot_grouped = ot_grouped.merge(
    baseline_stats, on=["CODE_STIF_LIGNE", "day_of_week"], how="left"
)

# Compute studentized residuals for each day’s NB_VALD
ot_grouped["std_val"] = ot_grouped["std_val"].replace(0, np.nan)
ot_grouped["studentized_residual"] = (
    (ot_grouped["NB_VALD"] - ot_grouped["mean_val"]) / ot_grouped["std_val"]
) * np.sqrt((ot_grouped["n"] + 1) / ot_grouped["n"])

# Define an anomaly threshold using the t-distribution
threshold = t.ppf(
    0.025, df=ot_grouped["n"] - 1
)  # 2.5% significance level for two-tailed test

# Flag potential strike days
ot_grouped["potential_strike"] = ot_grouped["studentized_residual"] < threshold

# Optionally, select only those rows that are flagged
possible_strike_days = ot_grouped[ot_grouped["potential_strike"]]

print("Potential strike days:")
possible_strike_days[
    ["JOUR", "CODE_STIF_LIGNE", "NB_VALD", "studentized_residual", "potential_strike"]
]

In [ ]:
possible_strike_days.groupby("JOUR")["potential_strike"].sum()

#### At daily level

In [ ]:
# Group by day and station, sum NB_VALD
ot_grouped = overground_transport.groupby(["JOUR"])["NB_VALD"].sum().reset_index()
ot_grouped = ot_grouped[
    (ot_grouped["JOUR"] >= "2020-09-01") & (ot_grouped["JOUR"] <= "2021-10-18")
]

# Ensure JOUR is a datetime type
ot_grouped["JOUR"] = pd.to_datetime(ot_grouped["JOUR"])

# Extract the day of week (0=Monday, 6=Sunday)
ot_grouped["day_of_week"] = ot_grouped["JOUR"].dt.dayofweek

# Compute baseline stats (mean, std, and count) per day_of_week and station
baseline_stats = (
    ot_grouped.groupby(["day_of_week"])["NB_VALD"]
    .agg(["mean", "std", "count"])
    .reset_index()
)
baseline_stats.rename(
    columns={"mean": "mean_val", "std": "std_val", "count": "n"}, inplace=True
)

# Merge the baseline stats back to the original DataFrame
ot_grouped = ot_grouped.merge(baseline_stats, on=["day_of_week"], how="left")

# Compute studentized residuals for each day’s NB_VALD
ot_grouped["std_val"] = ot_grouped["std_val"].replace(0, np.nan)
ot_grouped["studentized_residual"] = (
    (ot_grouped["NB_VALD"] - ot_grouped["mean_val"]) / ot_grouped["std_val"]
) * np.sqrt((ot_grouped["n"] + 1) / ot_grouped["n"])

# Define an anomaly threshold using the t-distribution
threshold = t.ppf(
    0.025, df=ot_grouped["n"] - 1
)  # 2.5% significance level for two-tailed test

# Flag potential strike days
ot_grouped["potential_strike"] = ot_grouped["studentized_residual"] < threshold

# Optionally, select only those rows that are flagged
possible_strike_days = ot_grouped[ot_grouped["potential_strike"]]

print("Potential strike days:")
possible_strike_days[["JOUR", "NB_VALD", "studentized_residual", "potential_strike"]]

In [ ]:
possible_strike_days.groupby("JOUR")["potential_strike"].sum()

## Merging underground and overground transport strike data